In [1]:
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=300)
L=LaurentSeriesRing(QQ,'x', default_prec=300)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.
def evaluate(expression,incomingvalue):
    expr=str(expression)
    output=sage_eval(expr,locals={'x':incomingvalue})
    return output

P=Primes()

def uniteList(lst):
    ln=len(lst)
    if ln<2:return lst
    answer=[]
    for j in [0..ln-1]:
        lj=lst[j]
        if (lj in answer)==False:
            answer=answer+[lj]
    return answer

def firsts(list):
    answer=[]
    for pair in list:
        answer=answer+[pair[0]]
    return answer

def prime(n): return P.unrank(n-1)

def unite(A,B):
    return set(A).union(B)

def factor_integer(n): #this is like "factor(integer)", but
    #the bases and exponents can be easily extracted.
    F=factor(n)
    return list(F)

def orderInteger(p,n):
    if n!=0:
        lst=n.digits(p)
        count=0
        k=0
        while lst[k]==0:
            count=count+1
            k=k+1
        return count
    if n==0:return(Infinity)

def order(p,fr):
    nn=numerator(QQ(fr))
    dn=denominator(QQ(fr))
    return orderInteger(p,nn)-orderInteger(p,dn)

def digitsum(p,n):return add(n.digits(p))

def digitsum(p,n):
    return sum(n.digits(p))

def rmnjntau(n):
    answer=0
    for k in [1..n-1]:
        answer=answer+(6*k-4*(n-k))*sigma(k,3)*sigma(n-k,5)
    answer=(5*sigma(n,3)+7*sigma(n,5))*n/12-35*answer
    return answer

def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])

def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer

def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))

def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))

def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)

def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))

def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)

def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)

def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # 
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.

def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)

def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]

def xjNew(n,m):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(m**3*2**6)**i
        return rt
    return xjNewStep(n-1,m)

def j(n,m):return L(xjNew(n,m)/x)

def jpowerConstantTerm(m,power):
    trun=j(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def jLeoPowerConstantTerm(m,power,multiplier):
    trun=jLeo(power,m,multiplier)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def polynomialCoefficient(n,poly):
    return poly.list()[n]

def leadingCoefficient(polynomial):
    dg=polynomial.degree()
    
    return polynomialCoefficient(dg,polynomial)

def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent

def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)

def firstfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.first())

def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())

def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)

def element(k,listo):
    if k>len(listo):return (print("bad k"))
    return listo[k-1]

def mobiusInverseOfFunction(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        f=f+moebius(dvk)*g(n/dvk)
    return f

def mobiusInverseOfList(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        gndvk=element(n/dvk,g)
        f=f+moebius(dvk)*gndvk
    return f

def drop_first_element(listo):
    lenth=len(listo)
    answer=[]
    for k in [1..lenth-1]:
        answer=answer+[listo[k]]
    return answer

def factorSeriesFromLists(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
    return nswr

def factorSeriesFromListsWithTracking(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
            print((n,k))
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
        print("answer length: ",len(nswr))
    return nswr

def EulerMultiplication(listo):
    # Lemma 2.11 from my 1998 Experimental Mathematics paper.
    # Here the set A in the lemma is {1, 2, ...} = Z^+.
    p_A=[1] # so p_A[0]=1 as on top of p. 266, column 2.
    def ex(n):return listo[n-1] # ex is the exponent on the factor 1-x^n: -f(n)/n.
    def f(n):return -n*ex(n)
    answer=[1]
    def f_A(k):
        summ=0
        dk=divisors(k)
        for d in dk: #Here is where A = Z^+ is used to drop a constraint on k.
            summ=summ+f(d)
        return summ
    for n in [1..len(listo)-1]:
        sm=0
        for k in [1..n]:
            sm=sm+f_A(k)*p_A[n-k]
        p_A=p_A+[sm/n]
    return p_A

def listToPoly(listo):
    lenth=len(listo)
    answer=0
    for k in [1..lenth]:
        cf=listo[k-1]
        answer=answer+cf*x^(k-1)
    return answer

def difference_table(lst):
    diffs=[]
    for k in [0..len(lst)-2]:
        diff=lst[k+1]-lst[k]
        diffs=diffs+[diff]
    return diffs

def last_element(listo):return listo[len(listo)-1]
    
#Broadhurst code from #A005148:

#? {nt=100; a=[1]; b=[1]; d=1; e=0; g=0; print(1); for(n=2, nt, c=48*(a[n-1]+g)+128*(d-32*e); 
#e=d; d=c; i=(n-1)\2; g=12*if(n%2==0, a[n/2]^2)+24*sum(j=1, i, 
#a[j]*a[n-j]); h=12*if(n%2==0, b[n/2]^2)+24*sum(j=1, i, b[j]*b[n-j]); f=(c+5*h)/n^2-g; a=concat(a, f); 
#b=concat(b, n*f); print(f))}

#produces this on the PARI online evaluator (I have inserted the commas):
br=[
1,
47,
2488,
138799,
7976456,
467232200,
27736348480,
1662803271215,
100442427373480,
6103747246289272,
372725876150863808,
22852464771010647496,
1405886026610765892544,
86741060172969340021952,
5365190340823180439326208,
332577246704242939511725615,
20655377769544663820919905000,
1285027807539621869480480977880,
80066610886753513409821525593280,
4995543732566526565060187887772024,
312067903389730540416319245145039936,
19516459352109724206910675815791735872,
1221787478073080268912138739833447254528,
76558881238278398609546573647116818306504,
4801399849802188285872546222298724299377856,
301358552889212442951924121355286655092791360,
18928524108186605379268259069278244869735006720,
1189719542605042010945455887482239233732751142080,
74824958481405101799295401923145498080031496317440,
4708731584940969251488540213411242070133095720768000,
296483323638911778793802123013217365155428610625064960,
18677571039055424502042574350078071038555962934810664495,
1177200955467256907707767829606512556434525730284672082280,
74229820742983998523807878655148660941364964757170232076440,
4682657672641000613276353688819373189604961982881761635174080,
295516785862704112676947743865736338547152307208873658542187480,
18656838683258040776726836797753969443154060448210951169536087360,
1178287550937265649491805466460363896744099593833261406542090821440,
74441259433548426510664621182339422182178689134172479673100078686720,
4704546876230537649051669928635037299315044055233418643313504347890040,
297410696380227510473584821926459754598587577997951261584830786025989440,
18807176292551896455842616399574167855948518855982280636468413444438841280,
1189632505858785415664268185396568316810012962868095237190924015678644805120,
75269436592700558660145646818728077669744495747378078929068356710829357904960,
4763606735739477078702262301306618196904330454342036172567804617626114845601280,
301550219357655322958904198748139655940272138707157414253528789096123355242370560,
19093491105382437947961430595496009051927469794600124607374594862297809973497425920,
1209229421833128214532165231904398024088456532579184673374765702204525386892709582280,
76599462222171488217469562807555444840329820375936645628428503967599842536403748392640,
4853249476279584943018752544135518205835823652569328104071808597099976302206777672382272,
307557161322636240130429378891312616501367843861094990590342588813640658716687276702796288,
19494049310445477564550759598016449348244789590449694853790210431694353584090244058674665536,
1235828235382992966112000337356874542591602729215314383988292324949451319922660010288507026944,
78359386541981263753587663605771220395173695760309511724942887619218844604408365062990458445312,
4969331730256147172835336645727281480799885420753251882375600240456687485460922105379475213365248,
315192836223790555211163444188645825759965673786927108224693473418739558121668975477385609064230080,
19995098780164839843271975968833674908892528077762766665489021901583285029365260596621852077438476800,
1268636632413268672729488723673519658349225434504337043331170274695193536624267473803997222986457986560,
80503572319005599896519802085748511261953758070993177770252662999751870126880338970620619030455487016960,
5109226023515231795174271806642022238278119546486922652746346463140324125835625762209193286894097317188096,
324306079896787161344357124063601140357997183120543361083780244695916059297903737976316852087325736124084224,
20587951238469144728875413572545165973124146169862711202444824611729329948630917798996527215521428049827442688,
1307155886598940172607280751052393343723376964347599361863473016335357927316478471270092962849033836059872428032,
83003437890819298061159220674050513337514687509034267017635549288525596646068731914307086031798152640655708675631,
5271297646867701243231620056348828996858362139183014747261830107569871776680492045283621280463257824831450361228904,
334803625831811164273307471398151444570380979015726639675077730502474540099263374815508109958714263034979870045590040,
21267301347069209180434546119316800827596753601228462222166318226579132663526299067285136882052633679235269979706997440,
1351085301766957831759285167329838127692235945761375100182242807152550787271816136969058195839695684514251809964546971800,
85842015014782910888737358332857326909867676530325317899274801087388530640486646412863052257542933472850576480889534912320,
5454594181103314395353884318297045062026694474478123129221277721208935931363965378785241897523306877686001373372245607417152,
346632363267943630179319892957146981236491202095299908808662717047804243504180203803669349935923567992160546743779715198734848,
22030211252839896655760044436342222596667157343136792244243996326108725373610974640845764035532176284454550965374859141605876696,
1400264005045227498693320089227035884825680282492704474103652886750979292873550448366299419926933438094105780187281431855082851904,
89010607580108363146057947796447019984406819117601908324673767705214888723143847657440514237648023072092581303730690468541358446272,
5658653431087770147783431973072442739707731119479989465299403213540757386414610090412270459203277370032300491403324236240673058231808,
359768282283136451938233183643880156808956764689158714079901896700007719423392215459057530585195812985027222576505521483746145927094080,
22875473496936829687402292014161718834412927763836095316924893494178776078421448526507427925257601899589958056800447532066174066611688960,
1454634188002381701560543002374509285000151863499255133979293536524595484779236558931041033869616374153934389337102738325693541500885040640,
92506668310411752697700027500541107599054510647853447997767226456825933551220718898889759216482942932408592031625495444638507054727309701120,
5883380647402071727818124358464357357061997088366913641247755161122708769301392216445412497947130792332367814389247083022141893681918673035640,
374209367072642703061850255891790471856520862005092473938011251040764312378697398309875157737634899606364054132972840806767260748645213681887040,
23803199270417251468737963122626707467816366171461192749563400557478169477201165362512851772159448598935076348960799452562007578125691327998310080,
1514217230105491006416286735192399705713778112569360921940868647785308212475854144353099514116567802127049424293324092083010142613089652094239352320,
96332413045374746200370571514086365134691986056054938357641223739177015618938978964850794817403946266783867698429445521201166592631737075434674509760,
6128968041798698464871983060993177701445048464359248049771600533082817745158394358756109989641756654653420473533147391458535031474858708357818557350400,
389970917973786025826826148232023527188983145106526374860563378061488842725970203994300386046448714269543873579140545202535929354547230426525139471434240,
24814546341676335548842726357075976008312393230217108797033854347814848781823641548406242672165084256308341602099263161325277248861481206883551599766753280,
1579097865621621078127966349210038916312383695762720347829578839794195077696910123713050602263102661325782769915208166009373762048848807388370046579798291520,
100493898862353192639599085772479279507605308104393463045761183781449039477596456154787934584841158653853713852389459365917827655606822211804400142188630504960,
6395841086583513721518708585123859851483827511052356442692606484016955654747071476807355907729334535301712240745743084597082858081715137457733426771133436244480,
407082422122816769108928976084224767774083074925265772652509091404070682745260601092117551854989868576150378265801324011663291789547097044490460289150388722831360,
25911536639811395996751656925990326528992266850810482361355730044302538357241007798389242049209348961586470395969302758332164645822911593670231053397678686836037120,
1649413547559894591700394303117536943729317363777978884992676745553087694293098678096611231647446457954863017134436134803435906819926459313995976199288693719229255680,
105000403824773770333649105789820480034335517728840968988334310643726824123735041790377778857595055184234699203779438530862092851011368656739154398110404364696677847040,
6684622341158487811999668275612452018461145981041862416511208589860402818472403829783593014653257049064920715626593324371966426230798803277918448269747031588460412436480,
425585443813049916737778209891287760908166309712249413374261870396364033955750883326966977701620958916811619741413201896787531939944010422188516605008297556440177053390280,
27096933232548903092613115617140470313230857234611666991627504070866204307527763605986822544358886296299818124185350044201435781986714935884426869011696899042312957811757760,
1725347275200146778788065707502165945462707429554800950293313680035690477290460889039398857128345130378453865634762702538349329424603778897699271642610584658725955102258591040,
109864008941653383497254010859226498595190791378492674731068039950918263737791469856053624392850573136737164626778125500188292591996295723267255817207828065983109119673561029120,
6996107097499132971946127571962655485323611430800974100070703369809020527942862074490096967477739069312314066266444140870911945979977377211097174725520015644037734486845682825024]

def broadhurst(n):return br[n-1]

def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

def delete_element(elt,ls):
    new=[]
    for j in ls:
        if j!=elt:new=new+[j]
    return new

def sortAndSmash(lst): #sorts and deletes repeats
    cropped=lst
    new=[]
    while len(cropped)>0:
        m=min(cropped)
        new=new+[m]
        cropped=delete_element(m,cropped)
    return(new)

print("okay")

okay


In [ ]:
#output snipped; I interrupted this after k = 94.
polydata=[]
numericalfactordata=[]
broadhurst_values=[]
tests=[]
import time
for power in [1..100]:
    start=time.time()
    data=[]
    for m in [3..3*power+3]:
        cst= jpowerConstantTerm(m,power)
        data=data+[[m,cst]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[power,rl]]
    nf=ZZ(numericalfactor(rl))
    difference=nf/24-nszfunction(power)
    tests=tests+[difference]
    finish=time.time()
    print("(power,difference,time): ",(power,difference,finish-start))
print("===============================================================================")
print("tests:")
print(tests)
#below was originally done in a separate cell
#import pickle
#wfile = open('/Users/barrybrent/26oct22no1.txt','wb')
#t = pickle.dumps(str(polydata))
#wfile.write(t)
#wfile.close()
---------------------------------------------------------------------------------------------------
polydata=[]
numericalfactordata=[]
broadhurst_values=[]
tests=[]
import time
for power in [95..100]:
    start=time.time()
    data=[]
    for m in [3..3*power+3]:
        cst=QQ(jpowerConstantTerm(m,power))
        data=data+[[m,cst]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[power,rl]]
    nf=ZZ(numericalfactor(rl))
    difference=nf/24-broadhurst(power)
    tests=tests+[difference]
    finish=time.time()
    print("(power,difference,time): ",(power,difference,finish-start))
print("===============================================================================")
print("tests:")
print(tests)
import pickle
wfile = open('/Users/barrybrent/26oct22no2.txt','wb')
t = pickle.dumps(str(polydata))
wfile.write(t)
wfile.close()
print("tests:")
print(tests)

In [2]:
a=[1,2,3];b=[4,5,6];c=a+b;print(c)

[1, 2, 3, 4, 5, 6]


In [6]:
import pickle
rsfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rsfile)
rsfile.close()
s=stripQuotationMarks(rs)

rtfile = open('/Users/barrybrent/26oct22no2.txt','rb')
rt=pickle.load(rtfile)
rtfile.close()
t=stripQuotationMarks(rt)

u=s+t

wufile = open('/Users/barrybrent/5dec22no1.txt','wb')
u = pickle.dumps(str(u))
wufile.write(u)

1968604

In [8]:
import pickle
rfile = open('/Users/barrybrent/5dec22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
print(len(s))

100


In [95]:
order(2,0)

+Infinity

In [3]:
import pickle
rsfile = open('/Users/barrybrent/5dec22no1.txt','rb')
rs=pickle.load(rsfile)
rsfile.close()
s=stripQuotationMarks(rs)

mp2=0
data=[]
p=2
tests2=[]
t_set=[]
no=[]
dk=2
skips=[]
#dkd=dk.digits(p)
for h in [0..len(s)-1]:
    k=s[h][0]
    if digitsum(p,k)==dk:
        poly=D(s[h][1])
        for m in [3..1003]:
                if mp2==mod(m,p^2): # < = > 4|m
                    polym=evaluate(poly,m)
                    o=order(p,polym)
                    a=order(p,m)
                   
                    t=(k*(a+6)+2-o)/4
                    if len(t_set)>0:
                        if t< max(t_set):no=no+[t]
                    t_set=t_set+[t]
                    cnt=catalan_number(t)
                    o2cnt=order(2,cnt)
                    tupl=(k,m,t,o2cnt)
                    tests2=tests2+[o2cnt]
                    print(tupl)
print();print("How many decreases in the t sequence?: ",len(no));print()
u2=uniteList(tests2);print("set union of 2-orders:",u2);print()
u4=uniteList(t_set);print("set union of t_set:");print(u4);print()
print("min(set union of 2-orders):",min(u4))
print("max(set union of 2-orders):",max(u4))
for j in [min(u4)..max(u4)]:
    cj=catalan_number(j)
    if order(2,cj)==1:
        if (j in u4)==False:
            skips=skips+[j]
print("skips:" ,skips)
# u4 in OEIS:
# Search: seq:2,4,5,8,9,11,16,17,19,23,32,33,35,39,47,64,65,67,71,79,95 
# Displaying 1-1 of 1 result found.	page 1
# A099628		Numbers m where m-th Catalan number A000108(m) = binomial(2m,m)/(m+1) 
# is divisible by 2 but not by 4

(3, 4, 2, 1)
(3, 8, 2, 1)
(3, 12, 2, 1)
(3, 16, 2, 1)
(3, 20, 2, 1)
(3, 24, 2, 1)
(3, 28, 2, 1)
(3, 32, 2, 1)
(3, 36, 2, 1)
(3, 40, 2, 1)
(3, 44, 2, 1)
(3, 48, 2, 1)
(3, 52, 2, 1)
(3, 56, 2, 1)
(3, 60, 2, 1)
(3, 64, 2, 1)
(3, 68, 2, 1)
(3, 72, 2, 1)
(3, 76, 2, 1)
(3, 80, 2, 1)
(3, 84, 2, 1)
(3, 88, 2, 1)
(3, 92, 2, 1)
(3, 96, 2, 1)
(3, 100, 2, 1)
(3, 104, 2, 1)
(3, 108, 2, 1)
(3, 112, 2, 1)
(3, 116, 2, 1)
(3, 120, 2, 1)
(3, 124, 2, 1)
(3, 128, 2, 1)
(3, 132, 2, 1)
(3, 136, 2, 1)
(3, 140, 2, 1)
(3, 144, 2, 1)
(3, 148, 2, 1)
(3, 152, 2, 1)
(3, 156, 2, 1)
(3, 160, 2, 1)
(3, 164, 2, 1)
(3, 168, 2, 1)
(3, 172, 2, 1)
(3, 176, 2, 1)
(3, 180, 2, 1)
(3, 184, 2, 1)
(3, 188, 2, 1)
(3, 192, 2, 1)
(3, 196, 2, 1)
(3, 200, 2, 1)
(3, 204, 2, 1)
(3, 208, 2, 1)
(3, 212, 2, 1)
(3, 216, 2, 1)
(3, 220, 2, 1)
(3, 224, 2, 1)
(3, 228, 2, 1)
(3, 232, 2, 1)
(3, 236, 2, 1)
(3, 240, 2, 1)
(3, 244, 2, 1)
(3, 248, 2, 1)
(3, 252, 2, 1)
(3, 256, 2, 1)
(3, 260, 2, 1)
(3, 264, 2, 1)
(3, 268, 2, 1)
(3, 272, 2, 1)
(3, 27

(9, 508, 8, 1)
(9, 512, 8, 1)
(9, 516, 8, 1)
(9, 520, 8, 1)
(9, 524, 8, 1)
(9, 528, 8, 1)
(9, 532, 8, 1)
(9, 536, 8, 1)
(9, 540, 8, 1)
(9, 544, 8, 1)
(9, 548, 8, 1)
(9, 552, 8, 1)
(9, 556, 8, 1)
(9, 560, 8, 1)
(9, 564, 8, 1)
(9, 568, 8, 1)
(9, 572, 8, 1)
(9, 576, 8, 1)
(9, 580, 8, 1)
(9, 584, 8, 1)
(9, 588, 8, 1)
(9, 592, 8, 1)
(9, 596, 8, 1)
(9, 600, 8, 1)
(9, 604, 8, 1)
(9, 608, 8, 1)
(9, 612, 8, 1)
(9, 616, 8, 1)
(9, 620, 8, 1)
(9, 624, 8, 1)
(9, 628, 8, 1)
(9, 632, 8, 1)
(9, 636, 8, 1)
(9, 640, 8, 1)
(9, 644, 8, 1)
(9, 648, 8, 1)
(9, 652, 8, 1)
(9, 656, 8, 1)
(9, 660, 8, 1)
(9, 664, 8, 1)
(9, 668, 8, 1)
(9, 672, 8, 1)
(9, 676, 8, 1)
(9, 680, 8, 1)
(9, 684, 8, 1)
(9, 688, 8, 1)
(9, 692, 8, 1)
(9, 696, 8, 1)
(9, 700, 8, 1)
(9, 704, 8, 1)
(9, 708, 8, 1)
(9, 712, 8, 1)
(9, 716, 8, 1)
(9, 720, 8, 1)
(9, 724, 8, 1)
(9, 728, 8, 1)
(9, 732, 8, 1)
(9, 736, 8, 1)
(9, 740, 8, 1)
(9, 744, 8, 1)
(9, 748, 8, 1)
(9, 752, 8, 1)
(9, 756, 8, 1)
(9, 760, 8, 1)
(9, 764, 8, 1)
(9, 768, 8, 1)
(9, 772, 8

(17, 60, 16, 1)
(17, 64, 16, 1)
(17, 68, 16, 1)
(17, 72, 16, 1)
(17, 76, 16, 1)
(17, 80, 16, 1)
(17, 84, 16, 1)
(17, 88, 16, 1)
(17, 92, 16, 1)
(17, 96, 16, 1)
(17, 100, 16, 1)
(17, 104, 16, 1)
(17, 108, 16, 1)
(17, 112, 16, 1)
(17, 116, 16, 1)
(17, 120, 16, 1)
(17, 124, 16, 1)
(17, 128, 16, 1)
(17, 132, 16, 1)
(17, 136, 16, 1)
(17, 140, 16, 1)
(17, 144, 16, 1)
(17, 148, 16, 1)
(17, 152, 16, 1)
(17, 156, 16, 1)
(17, 160, 16, 1)
(17, 164, 16, 1)
(17, 168, 16, 1)
(17, 172, 16, 1)
(17, 176, 16, 1)
(17, 180, 16, 1)
(17, 184, 16, 1)
(17, 188, 16, 1)
(17, 192, 16, 1)
(17, 196, 16, 1)
(17, 200, 16, 1)
(17, 204, 16, 1)
(17, 208, 16, 1)
(17, 212, 16, 1)
(17, 216, 16, 1)
(17, 220, 16, 1)
(17, 224, 16, 1)
(17, 228, 16, 1)
(17, 232, 16, 1)
(17, 236, 16, 1)
(17, 240, 16, 1)
(17, 244, 16, 1)
(17, 248, 16, 1)
(17, 252, 16, 1)
(17, 256, 16, 1)
(17, 260, 16, 1)
(17, 264, 16, 1)
(17, 268, 16, 1)
(17, 272, 16, 1)
(17, 276, 16, 1)
(17, 280, 16, 1)
(17, 284, 16, 1)
(17, 288, 16, 1)
(17, 292, 16, 1)
(17, 29

(20, 296, 19, 1)
(20, 300, 19, 1)
(20, 304, 19, 1)
(20, 308, 19, 1)
(20, 312, 19, 1)
(20, 316, 19, 1)
(20, 320, 19, 1)
(20, 324, 19, 1)
(20, 328, 19, 1)
(20, 332, 19, 1)
(20, 336, 19, 1)
(20, 340, 19, 1)
(20, 344, 19, 1)
(20, 348, 19, 1)
(20, 352, 19, 1)
(20, 356, 19, 1)
(20, 360, 19, 1)
(20, 364, 19, 1)
(20, 368, 19, 1)
(20, 372, 19, 1)
(20, 376, 19, 1)
(20, 380, 19, 1)
(20, 384, 19, 1)
(20, 388, 19, 1)
(20, 392, 19, 1)
(20, 396, 19, 1)
(20, 400, 19, 1)
(20, 404, 19, 1)
(20, 408, 19, 1)
(20, 412, 19, 1)
(20, 416, 19, 1)
(20, 420, 19, 1)
(20, 424, 19, 1)
(20, 428, 19, 1)
(20, 432, 19, 1)
(20, 436, 19, 1)
(20, 440, 19, 1)
(20, 444, 19, 1)
(20, 448, 19, 1)
(20, 452, 19, 1)
(20, 456, 19, 1)
(20, 460, 19, 1)
(20, 464, 19, 1)
(20, 468, 19, 1)
(20, 472, 19, 1)
(20, 476, 19, 1)
(20, 480, 19, 1)
(20, 484, 19, 1)
(20, 488, 19, 1)
(20, 492, 19, 1)
(20, 496, 19, 1)
(20, 500, 19, 1)
(20, 504, 19, 1)
(20, 508, 19, 1)
(20, 512, 19, 1)
(20, 516, 19, 1)
(20, 520, 19, 1)
(20, 524, 19, 1)
(20, 528, 19, 

(33, 304, 32, 1)
(33, 308, 32, 1)
(33, 312, 32, 1)
(33, 316, 32, 1)
(33, 320, 32, 1)
(33, 324, 32, 1)
(33, 328, 32, 1)
(33, 332, 32, 1)
(33, 336, 32, 1)
(33, 340, 32, 1)
(33, 344, 32, 1)
(33, 348, 32, 1)
(33, 352, 32, 1)
(33, 356, 32, 1)
(33, 360, 32, 1)
(33, 364, 32, 1)
(33, 368, 32, 1)
(33, 372, 32, 1)
(33, 376, 32, 1)
(33, 380, 32, 1)
(33, 384, 32, 1)
(33, 388, 32, 1)
(33, 392, 32, 1)
(33, 396, 32, 1)
(33, 400, 32, 1)
(33, 404, 32, 1)
(33, 408, 32, 1)
(33, 412, 32, 1)
(33, 416, 32, 1)
(33, 420, 32, 1)
(33, 424, 32, 1)
(33, 428, 32, 1)
(33, 432, 32, 1)
(33, 436, 32, 1)
(33, 440, 32, 1)
(33, 444, 32, 1)
(33, 448, 32, 1)
(33, 452, 32, 1)
(33, 456, 32, 1)
(33, 460, 32, 1)
(33, 464, 32, 1)
(33, 468, 32, 1)
(33, 472, 32, 1)
(33, 476, 32, 1)
(33, 480, 32, 1)
(33, 484, 32, 1)
(33, 488, 32, 1)
(33, 492, 32, 1)
(33, 496, 32, 1)
(33, 500, 32, 1)
(33, 504, 32, 1)
(33, 508, 32, 1)
(33, 512, 32, 1)
(33, 516, 32, 1)
(33, 520, 32, 1)
(33, 524, 32, 1)
(33, 528, 32, 1)
(33, 532, 32, 1)
(33, 536, 32, 

(36, 408, 35, 1)
(36, 412, 35, 1)
(36, 416, 35, 1)
(36, 420, 35, 1)
(36, 424, 35, 1)
(36, 428, 35, 1)
(36, 432, 35, 1)
(36, 436, 35, 1)
(36, 440, 35, 1)
(36, 444, 35, 1)
(36, 448, 35, 1)
(36, 452, 35, 1)
(36, 456, 35, 1)
(36, 460, 35, 1)
(36, 464, 35, 1)
(36, 468, 35, 1)
(36, 472, 35, 1)
(36, 476, 35, 1)
(36, 480, 35, 1)
(36, 484, 35, 1)
(36, 488, 35, 1)
(36, 492, 35, 1)
(36, 496, 35, 1)
(36, 500, 35, 1)
(36, 504, 35, 1)
(36, 508, 35, 1)
(36, 512, 35, 1)
(36, 516, 35, 1)
(36, 520, 35, 1)
(36, 524, 35, 1)
(36, 528, 35, 1)
(36, 532, 35, 1)
(36, 536, 35, 1)
(36, 540, 35, 1)
(36, 544, 35, 1)
(36, 548, 35, 1)
(36, 552, 35, 1)
(36, 556, 35, 1)
(36, 560, 35, 1)
(36, 564, 35, 1)
(36, 568, 35, 1)
(36, 572, 35, 1)
(36, 576, 35, 1)
(36, 580, 35, 1)
(36, 584, 35, 1)
(36, 588, 35, 1)
(36, 592, 35, 1)
(36, 596, 35, 1)
(36, 600, 35, 1)
(36, 604, 35, 1)
(36, 608, 35, 1)
(36, 612, 35, 1)
(36, 616, 35, 1)
(36, 620, 35, 1)
(36, 624, 35, 1)
(36, 628, 35, 1)
(36, 632, 35, 1)
(36, 636, 35, 1)
(36, 640, 35, 

(48, 440, 47, 1)
(48, 444, 47, 1)
(48, 448, 47, 1)
(48, 452, 47, 1)
(48, 456, 47, 1)
(48, 460, 47, 1)
(48, 464, 47, 1)
(48, 468, 47, 1)
(48, 472, 47, 1)
(48, 476, 47, 1)
(48, 480, 47, 1)
(48, 484, 47, 1)
(48, 488, 47, 1)
(48, 492, 47, 1)
(48, 496, 47, 1)
(48, 500, 47, 1)
(48, 504, 47, 1)
(48, 508, 47, 1)
(48, 512, 47, 1)
(48, 516, 47, 1)
(48, 520, 47, 1)
(48, 524, 47, 1)
(48, 528, 47, 1)
(48, 532, 47, 1)
(48, 536, 47, 1)
(48, 540, 47, 1)
(48, 544, 47, 1)
(48, 548, 47, 1)
(48, 552, 47, 1)
(48, 556, 47, 1)
(48, 560, 47, 1)
(48, 564, 47, 1)
(48, 568, 47, 1)
(48, 572, 47, 1)
(48, 576, 47, 1)
(48, 580, 47, 1)
(48, 584, 47, 1)
(48, 588, 47, 1)
(48, 592, 47, 1)
(48, 596, 47, 1)
(48, 600, 47, 1)
(48, 604, 47, 1)
(48, 608, 47, 1)
(48, 612, 47, 1)
(48, 616, 47, 1)
(48, 620, 47, 1)
(48, 624, 47, 1)
(48, 628, 47, 1)
(48, 632, 47, 1)
(48, 636, 47, 1)
(48, 640, 47, 1)
(48, 644, 47, 1)
(48, 648, 47, 1)
(48, 652, 47, 1)
(48, 656, 47, 1)
(48, 660, 47, 1)
(48, 664, 47, 1)
(48, 668, 47, 1)
(48, 672, 47, 

(66, 452, 65, 1)
(66, 456, 65, 1)
(66, 460, 65, 1)
(66, 464, 65, 1)
(66, 468, 65, 1)
(66, 472, 65, 1)
(66, 476, 65, 1)
(66, 480, 65, 1)
(66, 484, 65, 1)
(66, 488, 65, 1)
(66, 492, 65, 1)
(66, 496, 65, 1)
(66, 500, 65, 1)
(66, 504, 65, 1)
(66, 508, 65, 1)
(66, 512, 65, 1)
(66, 516, 65, 1)
(66, 520, 65, 1)
(66, 524, 65, 1)
(66, 528, 65, 1)
(66, 532, 65, 1)
(66, 536, 65, 1)
(66, 540, 65, 1)
(66, 544, 65, 1)
(66, 548, 65, 1)
(66, 552, 65, 1)
(66, 556, 65, 1)
(66, 560, 65, 1)
(66, 564, 65, 1)
(66, 568, 65, 1)
(66, 572, 65, 1)
(66, 576, 65, 1)
(66, 580, 65, 1)
(66, 584, 65, 1)
(66, 588, 65, 1)
(66, 592, 65, 1)
(66, 596, 65, 1)
(66, 600, 65, 1)
(66, 604, 65, 1)
(66, 608, 65, 1)
(66, 612, 65, 1)
(66, 616, 65, 1)
(66, 620, 65, 1)
(66, 624, 65, 1)
(66, 628, 65, 1)
(66, 632, 65, 1)
(66, 636, 65, 1)
(66, 640, 65, 1)
(66, 644, 65, 1)
(66, 648, 65, 1)
(66, 652, 65, 1)
(66, 656, 65, 1)
(66, 660, 65, 1)
(66, 664, 65, 1)
(66, 668, 65, 1)
(66, 672, 65, 1)
(66, 676, 65, 1)
(66, 680, 65, 1)
(66, 684, 65, 

(72, 424, 71, 1)
(72, 428, 71, 1)
(72, 432, 71, 1)
(72, 436, 71, 1)
(72, 440, 71, 1)
(72, 444, 71, 1)
(72, 448, 71, 1)
(72, 452, 71, 1)
(72, 456, 71, 1)
(72, 460, 71, 1)
(72, 464, 71, 1)
(72, 468, 71, 1)
(72, 472, 71, 1)
(72, 476, 71, 1)
(72, 480, 71, 1)
(72, 484, 71, 1)
(72, 488, 71, 1)
(72, 492, 71, 1)
(72, 496, 71, 1)
(72, 500, 71, 1)
(72, 504, 71, 1)
(72, 508, 71, 1)
(72, 512, 71, 1)
(72, 516, 71, 1)
(72, 520, 71, 1)
(72, 524, 71, 1)
(72, 528, 71, 1)
(72, 532, 71, 1)
(72, 536, 71, 1)
(72, 540, 71, 1)
(72, 544, 71, 1)
(72, 548, 71, 1)
(72, 552, 71, 1)
(72, 556, 71, 1)
(72, 560, 71, 1)
(72, 564, 71, 1)
(72, 568, 71, 1)
(72, 572, 71, 1)
(72, 576, 71, 1)
(72, 580, 71, 1)
(72, 584, 71, 1)
(72, 588, 71, 1)
(72, 592, 71, 1)
(72, 596, 71, 1)
(72, 600, 71, 1)
(72, 604, 71, 1)
(72, 608, 71, 1)
(72, 612, 71, 1)
(72, 616, 71, 1)
(72, 620, 71, 1)
(72, 624, 71, 1)
(72, 628, 71, 1)
(72, 632, 71, 1)
(72, 636, 71, 1)
(72, 640, 71, 1)
(72, 644, 71, 1)
(72, 648, 71, 1)
(72, 652, 71, 1)
(72, 656, 71, 

(96, 380, 95, 1)
(96, 384, 95, 1)
(96, 388, 95, 1)
(96, 392, 95, 1)
(96, 396, 95, 1)
(96, 400, 95, 1)
(96, 404, 95, 1)
(96, 408, 95, 1)
(96, 412, 95, 1)
(96, 416, 95, 1)
(96, 420, 95, 1)
(96, 424, 95, 1)
(96, 428, 95, 1)
(96, 432, 95, 1)
(96, 436, 95, 1)
(96, 440, 95, 1)
(96, 444, 95, 1)
(96, 448, 95, 1)
(96, 452, 95, 1)
(96, 456, 95, 1)
(96, 460, 95, 1)
(96, 464, 95, 1)
(96, 468, 95, 1)
(96, 472, 95, 1)
(96, 476, 95, 1)
(96, 480, 95, 1)
(96, 484, 95, 1)
(96, 488, 95, 1)
(96, 492, 95, 1)
(96, 496, 95, 1)
(96, 500, 95, 1)
(96, 504, 95, 1)
(96, 508, 95, 1)
(96, 512, 95, 1)
(96, 516, 95, 1)
(96, 520, 95, 1)
(96, 524, 95, 1)
(96, 528, 95, 1)
(96, 532, 95, 1)
(96, 536, 95, 1)
(96, 540, 95, 1)
(96, 544, 95, 1)
(96, 548, 95, 1)
(96, 552, 95, 1)
(96, 556, 95, 1)
(96, 560, 95, 1)
(96, 564, 95, 1)
(96, 568, 95, 1)
(96, 572, 95, 1)
(96, 576, 95, 1)
(96, 580, 95, 1)
(96, 584, 95, 1)
(96, 588, 95, 1)
(96, 592, 95, 1)
(96, 596, 95, 1)
(96, 600, 95, 1)
(96, 604, 95, 1)
(96, 608, 95, 1)
(96, 612, 95, 

In [21]:
# Re-do in a way that drops the reasoning about "skips".
import time
start=time.time()
bottomley=[]
for k in [1..20000]:
    cnk=catalan_number(k)
    if order(2,cnk)==1:
        bottomley=bottomley+[k]
finish=time.time()
print((finish-start)/60)
print(len(bottomley))
print(bottomley)

0.38546936909357704
103
[2, 4, 5, 8, 9, 11, 16, 17, 19, 23, 32, 33, 35, 39, 47, 64, 65, 67, 71, 79, 95, 128, 129, 131, 135, 143, 159, 191, 256, 257, 259, 263, 271, 287, 319, 383, 512, 513, 515, 519, 527, 543, 575, 639, 767, 1024, 1025, 1027, 1031, 1039, 1055, 1087, 1151, 1279, 1535, 2048, 2049, 2051, 2055, 2063, 2079, 2111, 2175, 2303, 2559, 3071, 4096, 4097, 4099, 4103, 4111, 4127, 4159, 4223, 4351, 4607, 5119, 6143, 8192, 8193, 8195, 8199, 8207, 8223, 8255, 8319, 8447, 8703, 9215, 10239, 12287, 16384, 16385, 16387, 16391, 16399, 16415, 16447, 16511, 16639, 16895, 17407, 18431]


In [89]:
# Re-do in a way that drops the reasoning about "skips".

bottomleyList=[]
for k in [0..20000]:
    cnk=catalan_number(k)
    if order(2,cnk)==1:
        bottomleyList=bottomleyList+[k]
def bottomley(n):return bottomleyList[n-1]
def inverse_bottomley(t):
    for n in [1..len(bottomleyList)]:
        if bottomley(n)==t:return n
import pickle
rsfile = open('/Users/barrybrent/5dec22no1.txt','rb')
rs=pickle.load(rsfile)
rsfile.close()
s=stripQuotationMarks(rs)

mp2=0
data=[]
p=2
t_set=[]
no=[]
dk=2
bt_list=[]
tuples=[]
#dkd=dk.digits(p)
n=0
for h in [0..len(s)-1]:
    k=s[h][0]
    if digitsum(p,k)==dk:
        n=n+1
        poly=D(s[h][1])
        for m in [3..103]:
            if mp2==mod(m,p^2): # < = > 4|m
                polym=evaluate(poly,m)
                o=order(p,polym)
                a=order(p,m)
                t=(k*(a+6)+2-o)/4
                for nn in [1..100]:
                    if bottomley(nn)==t:ibt=nn # so ibt =inverse_bottomley(t)
                bt_list=bt_list+[ibt]
                #print((k,m,t,bottomley(bt),bt))
                
                # we see below that ibt = n, so that
                # bottomley(ibt) = bottomley(n);
                # therefore: t = bottomley(n).
                tupl=(k,n-ibt,t-bottomley(n))
                tuples=tuples+[tupl]
                print(tupl)
print()
print(uniteList(bt_list));
print()
ul=uniteList(tuples)
for n in [0..len(ul)-1]:
    tupl=ul[n]
    print(tupl)

(3, 0, 0)
(3, 0, 0)
(3, 0, 0)
(3, 0, 0)
(3, 0, 0)
(3, 0, 0)
(3, 0, 0)
(3, 0, 0)
(3, 0, 0)
(3, 0, 0)
(3, 0, 0)
(3, 0, 0)
(3, 0, 0)
(3, 0, 0)
(3, 0, 0)
(3, 0, 0)
(3, 0, 0)
(3, 0, 0)
(3, 0, 0)
(3, 0, 0)
(3, 0, 0)
(3, 0, 0)
(3, 0, 0)
(3, 0, 0)
(3, 0, 0)
(5, 0, 0)
(5, 0, 0)
(5, 0, 0)
(5, 0, 0)
(5, 0, 0)
(5, 0, 0)
(5, 0, 0)
(5, 0, 0)
(5, 0, 0)
(5, 0, 0)
(5, 0, 0)
(5, 0, 0)
(5, 0, 0)
(5, 0, 0)
(5, 0, 0)
(5, 0, 0)
(5, 0, 0)
(5, 0, 0)
(5, 0, 0)
(5, 0, 0)
(5, 0, 0)
(5, 0, 0)
(5, 0, 0)
(5, 0, 0)
(5, 0, 0)
(6, 0, 0)
(6, 0, 0)
(6, 0, 0)
(6, 0, 0)
(6, 0, 0)
(6, 0, 0)
(6, 0, 0)
(6, 0, 0)
(6, 0, 0)
(6, 0, 0)
(6, 0, 0)
(6, 0, 0)
(6, 0, 0)
(6, 0, 0)
(6, 0, 0)
(6, 0, 0)
(6, 0, 0)
(6, 0, 0)
(6, 0, 0)
(6, 0, 0)
(6, 0, 0)
(6, 0, 0)
(6, 0, 0)
(6, 0, 0)
(6, 0, 0)
(9, 0, 0)
(9, 0, 0)
(9, 0, 0)
(9, 0, 0)
(9, 0, 0)
(9, 0, 0)
(9, 0, 0)
(9, 0, 0)
(9, 0, 0)
(9, 0, 0)
(9, 0, 0)
(9, 0, 0)
(9, 0, 0)
(9, 0, 0)
(9, 0, 0)
(9, 0, 0)
(9, 0, 0)
(9, 0, 0)
(9, 0, 0)
(9, 0, 0)
(9, 0, 0)
(9, 0, 0)
(9, 0, 0)
(9, 0, 0)
(9, 0, 0)


In [49]:
bottomley(1)

2

In [91]:
# Re-do in a way that drops the reasoning about "skips".

bottomleyList=[]
for k in [0..20000]:
    cnk=catalan_number(k)
    if order(2,cnk)==1:
        bottomleyList=bottomleyList+[k]
def bottomley(n):return bottomleyList[n-1]
def inverse_bottomley(t):
    for n in [1..len(bottomleyList)]:
        if bottomley(n)==t:return n
import pickle
rsfile = open('/Users/barrybrent/5dec22no1.txt','rb')
rs=pickle.load(rsfile)
rsfile.close()
s=stripQuotationMarks(rs)

mp2=0
differences=[]
p=2
t_set=[]
no=[]
dk=2
bt_list=[]
tuples=[]
#dkd=dk.digits(p)
n=0
for h in [0..len(s)-1]:
    k=s[h][0]
    if digitsum(p,k)==dk:
        n=n+1
        poly=D(s[h][1])
        for m in [3..103]:
            if mp2==mod(m,p^2): # < = > 4|m
                polym=evaluate(poly,m)
                o=order(p,polym)
                a=order(p,m)
                t=(k*(a+6)+2-o)/4
                diff=t-bottomley(n)
                differences=differences+[diff]
                tupl=(k,diff)
                print(tupl)
print()
print(differences)

(3, 0)
(3, 0)
(3, 0)
(3, 0)
(3, 0)
(3, 0)
(3, 0)
(3, 0)
(3, 0)
(3, 0)
(3, 0)
(3, 0)
(3, 0)
(3, 0)
(3, 0)
(3, 0)
(3, 0)
(3, 0)
(3, 0)
(3, 0)
(3, 0)
(3, 0)
(3, 0)
(3, 0)
(3, 0)
(5, 0)
(5, 0)
(5, 0)
(5, 0)
(5, 0)
(5, 0)
(5, 0)
(5, 0)
(5, 0)
(5, 0)
(5, 0)
(5, 0)
(5, 0)
(5, 0)
(5, 0)
(5, 0)
(5, 0)
(5, 0)
(5, 0)
(5, 0)
(5, 0)
(5, 0)
(5, 0)
(5, 0)
(5, 0)
(6, 0)
(6, 0)
(6, 0)
(6, 0)
(6, 0)
(6, 0)
(6, 0)
(6, 0)
(6, 0)
(6, 0)
(6, 0)
(6, 0)
(6, 0)
(6, 0)
(6, 0)
(6, 0)
(6, 0)
(6, 0)
(6, 0)
(6, 0)
(6, 0)
(6, 0)
(6, 0)
(6, 0)
(6, 0)
(9, 0)
(9, 0)
(9, 0)
(9, 0)
(9, 0)
(9, 0)
(9, 0)
(9, 0)
(9, 0)
(9, 0)
(9, 0)
(9, 0)
(9, 0)
(9, 0)
(9, 0)
(9, 0)
(9, 0)
(9, 0)
(9, 0)
(9, 0)
(9, 0)
(9, 0)
(9, 0)
(9, 0)
(9, 0)
(10, 0)
(10, 0)
(10, 0)
(10, 0)
(10, 0)
(10, 0)
(10, 0)
(10, 0)
(10, 0)
(10, 0)
(10, 0)
(10, 0)
(10, 0)
(10, 0)
(10, 0)
(10, 0)
(10, 0)
(10, 0)
(10, 0)
(10, 0)
(10, 0)
(10, 0)
(10, 0)
(10, 0)
(10, 0)
(12, 0)
(12, 0)
(12, 0)
(12, 0)
(12, 0)
(12, 0)
(12, 0)
(12, 0)
(12, 0)
(12, 0)
(12, 0)
(12, 0)
(12,